In [3]:
import pandas as pd
import numpy as np
from  pathlib import Path
from pyspark.sql import SparkSession, Row
import pyspark.sql.functions as F
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
from datetime import datetime, date
from IPython.display import display

# NOTE: to start a spark session, Java JDK must be installed.
spark = SparkSession.builder.getOrCreate()

import ipykernel
import ipywidgets as wg
import string
from uuid import uuid4


23/02/11 19:13:48 WARN Utils: Your hostname, workPC resolves to a loopback address: 127.0.1.1; using 192.168.1.170 instead (on interface enp3s0)
23/02/11 19:13:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/11 19:13:49 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [ ]:
#refactoring:

class Experiment:
    def __init__(self, id, description):
        self.id = id
        self.hash = str(uuid4())
        self.plate_ids = []
        self.description = description
        self.plate = Plate
        self.plate_objs = []
        self.creation_date = str(datetime())

        
    def listPlates(self):
        print(self.plate_objs)
        
    def createPlate(self, id=None, description=None, path_to_file =None, cols=None):
        if id == None:
            max = 1
            for x in self.plate_ids:
                max = int(x.split('plate')[-1])
                if int(x.split('plate')[-1])>max:
                    max = int(x.split('plate')[-1])
            id = 'plate' + str(max)
        self.plate_ids.append(id)
        if path_to_file:
            self.plate_objs.append(self.plate(id, description, path_to_file, cols))
            if not cols:
                #proocess all cols
            else:
                #some cols and time
                

        else:
            self.plate_objs.append(self.plate(id, description))
        
        
        
        
        
class Plate:
    def __init__(self, id, description=None, raw_df = None):
        self.id = id
        self.hash = str(uuid4())
        self.well_ids = []
        self.description = description
        self.well = Well
        self.well_objs = []
        self.raw_df = []
        self.creation_date = str(datetime())

        
        
    # def listWells
    
    
    def createWell(self, well_id=None, description = None, activity=False, df=None, time=None, absorbance=None):
        self.well_ids.append(well_id)
        self.well_objs.append(self.well(well_id, description, activity, df, time, absorbance))
    
    
    
    
    
class Well:
    def __init__(self, id, description=None, activity=False, df=None, time=None, absorbance=None):
        self.id = id
        self.hash = str(uuid4())
        self.description = description
        self.creation_date = str(datetime())
        self.active = activity
        self.df = df
        self.time = time
        self.absorbance = absorbance
    
        
    
    

In [7]:
#draft/alpha version (overly complicated)

class PlateExperiment:
    def __init__(self, id):
        self.id = id
        self.plate_ids = []
        self.virtual_plates = []
        self.raw_dfs = []
        self.total_wells = []
        self.active_wells = [] #make objects
        self.y_dims = []
        self.x_dims = []
        self.well = Well
        self.well_objs = []
        self.meta_df=[]
    # def __str__(self):
    #     print(f"ExperimentID: {self.id}, Plates in this experimient: {self.plate_ids}" )
    def y_list(self, y_dim):
        y_array = [string.ascii_uppercase[x] for x in range(y_dim)[:26]]
        if y_dim > 27:
            y_array += [string.ascii_uppercase[0] + string.ascii_uppercase[y] for y in range(y_dim-26)]
        return y_array

    def x_list(self, x_dim):
        return [y for y in range(x_dim+1)][1:]
    
    def create_null_data(self, x_dim, y_dim):
        out_ar = []
        for y in self.y_list(y_dim):
            row = [y]
            for x in self.x_list(x_dim):
                row.append(f"{y}{x}")
                tup_row = tuple(row)
            out_ar.append(tup_row)
        return out_ar

    def create_list_wells(self, x_dim, y_dim):
        out_ar = []
        for y in self.y_list(y_dim):
            for x in self.x_list(x_dim):
                out_ar.append(f"{y}{x}")
        return out_ar
        
    def addEmptyPlate(self, x_dim, y_dim, plate_id = None):
        if plate_id in self.plate_ids:
            print(f"Plate ID: {plate_id} already exists. Please choose another one or edit an existing plate.")
        else:
            
            if plate_id == None:
                max = 0
                for x in self.plate_ids:
                    if 'Empty' in x[:5]:
                        if int(x[5:]) > max:
                            max = int(x[5:])
                self.plate_ids.append(f"Empty{max+1}") 
            else:
                self.plate_ids.append(plate_id)
            
            self.x_dims.append(x_dim)
            self.y_dims.append(y_dim)
            self.total_wells.append(self.create_list_wells(x_dim, y_dim))
            self.active_wells.append([])
            self.well_objs.append([])
            self.raw_dfs.append([])
            self.virtual_plates.append(spark.createDataFrame(data=self.create_null_data(self.x_dims[-1], self.y_dims[-1]),schema=(["Letter"] + (list(map(str, self.x_list(x_dim)))))))
            print(f'Added plate {self.plate_ids[-1]}:\n{self.virtual_plates[-1]}')
            self.virtual_plates[-1].show()
    
    # def deletePlate
    
    
    # def addDataManually(self, plate_id, well_id):
        
    # add to well objects
    def addMetaFromFile(self, path_to_file, x_dim=12, y_dim=8, plate_id = "new_plate"):
        pandas_df = pd.read_excel(Path(path_to_file))
        df = spark.createDataFrame(pandas_df)
        print('Loaded file...')
        out = []
        if plate_id == "new_plate":
            new_id = path_to_file.split('/')[-1]
            plate_id = new_id
        
        if plate_id in self.plate_ids:
            self.x_dims[self.plate_ids.index(plate_id)] = x_dim
            self.y_dims[self.plate_ids.index(plate_id)] = y_dim

            self.active_wells[self.plate_ids.index(plate_id)] = ([df.select(df.columns[0]).rdd.map(lambda x : x[0]).collect()])
            self.total_wells[self.plate_ids.index(plate_id)] = (self.create_list_wells(x_dim, y_dim))
            self.virtual_plates[self.plate_ids.index(plate_id)] = (spark.createDataFrame(data=self.create_null_data(self.x_dims[-1], self.y_dims[-1]),schema=(["Letter"] + (list(map(str, self.x_list(x_dim)))))))
            wells_array = []
            
            for x in self.active_wells[-1]:
                wells_array.append(self.well(x, df = None, metadf = df))
                
            self.well_objs[self.plate_ids.index(plate_id)] = wells_array
            print(f"Edited plate {plate_id} meta-data.")
            df.show()
        else:
            self.plate_ids.append(plate_id)
            self.x_dims.append(x_dim)
            self.y_dims.append(y_dim)
            self.active_wells.append([df.select(df.columns[0]).rdd.map(lambda x : x[0]).collect()])
            self.total_wells.append(self.create_list_wells(x_dim, y_dim))
            self.virtual_plates.append(spark.createDataFrame(data=self.create_null_data(self.x_dims[-1], self.y_dims[-1]),schema=(["Letter"] + (list(map(str, self.x_list(x_dim)))))))
            wells_array = []
            
            for x in self.active_wells[-1]:
                wells_array.append(self.well(x, df = None, metadf = df))
                
            self.well_objs.append(wells_array)
            print(f"Added plate {plate_id} meta-data.")
            df.show()
                
                
            
            
        
    
    def listPlates(self):
        for x in self.plate_ids:
            print(f"{x}\n")
          
    def getPlate(self, plate_id):
        
        if plate_id in self.plate_ids:
            print(f"{plate_id}:\nActive wells:\n{self.active_wells[self.plate_ids.index(plate_id)]}")
            self.virtual_plates[self.plate_ids.index(plate_id)].show()
            return (self.virtual_plates[self.plate_ids.index(plate_id)])
        else:
            print('Plate ID incorrect or invalid.')
            
        
            
        
    
    def setActiveWells(self, plate_id, wells = ['all']):
        if wells == ['all']:
            self.active_wells[self.plate_ids.index(plate_id)] = self.total_wells[self.plate_ids.index(plate_id)]
        else:
            for x in self.total_wells[self.plate_ids.index(plate_id)]:
                if x in wells:
                    self.active_wells[self.plate_ids.index(plate_id)].append(x)
        print(f"Plate {self.plate_ids[self.plate_ids.index(plate_id)]} active wells:\n{self.active_wells[self.plate_ids.index(plate_id)]}")
                
    

    
    
        # add well objects in the matrix
    def addExcelMeasurment(self, path_to_file, plate_id = None, x_dim = 12, y_dim = 8):
        if plate_id == None:
                id = str(path_to_file).split('/')[-1]
                plate_id = id
        if id in self.plate_ids:
            
            pandas_df = pd.read_excel(Path(path_to_file))
            df = spark.createDataFrame(pandas_df)
            self.x_dims[self.plate_ids.index(plate_id)] = x_dim
            self.y_dims[self.plate_ids.index(plate_id)] = y_dim
            wells_array = []
            active_wells = []
            for x in self.create_list_wells(self.x_dims[-1], self.y_dims[-1]) and df.columns[1:]:
                active_wells.append(x)
                well_df = df.select(F.col(f'Time_min'), F.col(f'{x}'))
                wells_array.append(self.well(well_id = str(x), df=well_df))
            self.active_wells[self.plate_ids.index(plate_id)] = active_wells
            self.total_wells[self.plate_ids.index(plate_id)] = (self.create_list_wells(x_dim, y_dim))
            self.virtual_plates[self.plate_ids.index(plate_id)] = (spark.createDataFrame(data=self.create_null_data(self.x_dims[-1], self.y_dims[-1]),schema=(["Letter"] + (list(map(str, self.x_list(x_dim)))))))
                
            self.well_objs[self.plate_ids.index(plate_id)] = wells_array
            self.raw_dfs[self.plate_ids.index(plate_id)] = df
            print(f"Edited plate {plate_id} measurements.")
            print(f"Added file {id} to ID: {id}\nRaw Plate DataFrame:\n{self.raw_dfs[-1]}")
            self.raw_dfs[self.plate_ids.index(plate_id)].show()
            print(f"Wells {self.active_wells[self.plate_ids.index(plate_id)]} added. Access meta-data and measurements via the wells_objs variable per plate or make use of getWellData function.")
            
        else:
            self.x_dims.append(x_dim)
            if plate_id == None:
                self.plate_ids.append(id)
            else:
                self.plate_ids.append(plate_id)
            
            self.y_dims.append(y_dim)
            self.total_wells.append(self.create_list_wells(x_dim, y_dim))        
            pandas_df = pd.read_excel(Path(path_to_file))
            df = spark.createDataFrame(pandas_df)
                    #upload to DB
            self.raw_dfs.append(df)
            wells_array = []
            active_wells = []
            for x in self.create_list_wells(self.x_dims[-1], self.y_dims[-1]) and df.columns[1:]:
                active_wells.append(x)
                well_df = df.select(F.col(f'Time_min'), F.col(f'{x}'))
                wells_array.append(self.well(well_id = str(x), df=well_df))
                # self.absorbance.append(df.select(f"{x}").rdd.map(lambda x : x[0]).collect())
            self.well_objs.append(wells_array)
            self.active_wells.append(active_wells)
            self.virtual_plates.append(spark.createDataFrame(data=self.create_null_data(self.x_dims[-1], self.y_dims[-1]),schema=(["Letter"] + (list(map(str, self.x_list(x_dim)))))))   
            
            print(f"Added file {id} as ID: {self.plate_ids[-1]}\nRaw Plate DataFrame:\n{self.raw_dfs[-1]}")
            self.raw_dfs[-1].show()
            print(f"Wells {self.active_wells[-1]} added. Access via the wells_objs variable per plate or make use of getWellData function.")
            
    def getWellData(self, plate_id, wells = ['all']):
        out = []
        for x in self.well_objs[self.plate_ids.index(plate_id)]:    
            if str(x.well_id) in wells:
                print(x.well_id)
                if x.df:
                    print("Measurement data:")
                    x.df.show()
                else:
                    print('No measurement data.')
                if x.meta:
                    print("Meta-data:")
                    x.meta.show()
                else:
                    print('No meta-data.')
                out.append(x)
            elif wells == ['all']:
                print(x.well_id)
                if x.df:
                    print("Measurement data:")
                    x.df.show()
                else:
                    print('No measurement data.')
                if x.meta:
                    print("Meta-data:")
                    x.meta.show()
                else:
                    print('No meta-data.')
                out.append(x)
        return out
        
                
    def dfToArrays(self, plate_id, cols = ["all"]):#link to pyenzyme
        out = []
        if cols == ["all"]:  
            for x in self.active_wells[self.plate_ids.index(plate_id)]:
                print(f"Column {x}:\n{self.raw_dfs[self.plate_ids.index(plate_id)].select(x).rdd.map(lambda x : x[0]).collect()}")
                out.append(self.raw_dfs[self.plate_ids.index(plate_id)].select(f"{x}").rdd.map(lambda x : x[0]).collect())
        else:
            for x in cols:
                print(f"Column {x}:\n{self.raw_dfs[self.plate_ids.index(plate_id)].select(x).rdd.map(lambda x : x[0]).collect()}")
                out.append(self.raw_dfs[self.plate_ids.index(plate_id)].select(f"{x}").rdd.map(lambda x : x[0]).collect())
        return out
            

class Well:
    def __init__ (self, well_id = None, df =None, metadf = None):
        #add meta-data
        self.well_id = well_id
        self.df = df
        self.meta = metadf
    def __str__(self):
        return f'{self.well_id}'
        
        
    def getData(self):
        return f"Well {self.id}\nTime:\n" and self.df.show()
        


In [8]:
exp1 = PlateExperiment(id= 1)


In [9]:
exp1.addMetaFromFile("./LADHData/output_plate1.xlsx")
exp1.addExcelMeasurment("./LADHData/NAD1.xlsx")

/home/cdb/.local/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/home/cdb/.local/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


Loaded file...
Added plate output_plate1.xlsx meta-data.
+----------+-------------------+----+-----+
|Unnamed: 0|               rate|NADH|  Pyr|
+----------+-------------------+----+-----+
|        A1|0.09703995957556694| 0.5| 40.5|
|        A2| 0.1036444048016007| 0.5| 40.5|
|        A3|                NaN| 0.5| 40.5|
|        B1|  0.105380061864448| 0.5| 37.8|
|        B2|0.09787776226298234| 0.5| 37.8|
|        B3| 0.1039922551535382| 0.5| 37.8|
|        C1|0.09984372480189613| 0.5| 27.0|
|        C2| 0.1098075978147142| 0.5| 27.0|
|        C3|0.09914831086691459| 0.5| 27.0|
|        D1|0.09644437300363148| 0.5| 13.5|
|        D2|0.09964015797548002| 0.5| 13.5|
|        D3| 0.1023223968600116| 0.5| 13.5|
|        E1|0.06841155163084021| 0.5| 6.75|
|        E2|0.06591782004740387| 0.5| 6.75|
|        E3|0.07406108808410754| 0.5| 6.75|
|        F1|0.03533653824357928| 0.5|4.455|
|        F2|0.04286823419274025| 0.5|4.455|
|        F3|0.04641613917429578| 0.5|4.455|
|        G1|0.04531

/home/cdb/.local/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/home/cdb/.local/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


Added file NAD1.xlsx as ID: NAD1.xlsx
Raw Plate DataFrame:
DataFrame[Time_min: double, A2: double, A3: double, B1: double, B2: double, B3: double, C1: double, C2: double, C3: double, D1: double, D2: double, D3: double, E1: double, E2: double, E3: double, F1: double, F2: double, F3: double, G1: double, G2: double, G3: double]
+-------------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|           Time_min|               A2|               A3|               B1|               B2|               B3|               C1|               C2|               C3|               D1|               D2|               D3|               E1|               E2|               E3|               F1|

In [12]:
exp1.getWellData("output_plate1.xlsx")

['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3', 'D1', 'D2', 'D3', 'E1', 'E2', 'E3', 'F1', 'F2', 'F3', 'G1', 'G2', 'G3', 'A4', 'A5', 'A6', 'B4', 'B5', 'B6', 'C4', 'C5', 'C6', 'D4', 'D5', 'D6', 'E4', 'E5', 'E6', 'F4', 'F5', 'F6', 'G4', 'G5', 'G6', 'A7', 'A8', 'A9', 'B7', 'B8', 'B9', 'C7', 'C8', 'C9', 'D7', 'D8', 'D9', 'E7', 'E8', 'E9', 'F7', 'F8', 'F9', 'G7', 'G8', 'G9', 'A10', 'A11', 'A12', 'B10', 'B11', 'B12', 'C10', 'C11', 'C12', 'D10', 'D11', 'D12', 'E10', 'E11', 'E12', 'F10', 'F11', 'F12', 'G10', 'G11', 'G12']
No measurement data.
Meta-data:
+----------+-------------------+----+-----+
|Unnamed: 0|               rate|NADH|  Pyr|
+----------+-------------------+----+-----+
|        A1|0.09703995957556694| 0.5| 40.5|
|        A2| 0.1036444048016007| 0.5| 40.5|
|        A3|                NaN| 0.5| 40.5|
|        B1|  0.105380061864448| 0.5| 37.8|
|        B2|0.09787776226298234| 0.5| 37.8|
|        B3| 0.1039922551535382| 0.5| 37.8|
|        C1|0.09984372480189613| 0.5| 27.0|
| 

In [88]:
exp1.getPlate("NAD1.xlsx")

Plate ID incorrect or invalid.


In [89]:

exp1.addPlateFromExcel("./LADHData/NAD1.xlsx")
exp1.addPlateFromExcel("./LADHData/NAD2.xlsx")

/home/cdb/.local/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/home/cdb/.local/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


Added file NAD1.xlsx as ID: NAD1.xlsx
Raw Plate DataFrame:
DataFrame[Time_min: double, A2: double, A3: double, B1: double, B2: double, B3: double, C1: double, C2: double, C3: double, D1: double, D2: double, D3: double, E1: double, E2: double, E3: double, F1: double, F2: double, F3: double, G1: double, G2: double, G3: double]
+-------------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|           Time_min|               A2|               A3|               B1|               B2|               B3|               C1|               C2|               C3|               D1|               D2|               D3|               E1|               E2|               E3|               F1|

/home/cdb/.local/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/home/cdb/.local/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


Added file NAD2.xlsx as ID: NAD2.xlsx
Raw Plate DataFrame:
DataFrame[Time_min: double, A4: double, A5: double, A6: double, B4: double, B5: double, B6: double, C4: double, C5: double, C6: double, D4: double, D5: double, D6: double, E4: double, E5: double, E6: double, F4: double, F5: double, F6: double, G4: double, G5: double, G6: double]
+-------------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|           Time_min|               A4|               A5|               A6|               B4|               B5|               B6|               C4|               C5|               C6|               D4|               D5|               D6|               E4|      

In [90]:
exp1.listPlates()

Empty1

NAD1.xlsx

NAD2.xlsx



In [ ]:
# def null_plate_df(x_dim, y_dim):
#     return spark.createDataFrame(data=create_null_data(x_dim, y_dim),schema=(["Letter"] + (list(map(str, x_list(x_dim))))))


# null_plate_df(4,4).show()

Create a microtiter plate setup (for PyEnzyme add replicate func   ):
    3. Add support for automated metadata loading from standard format (instead of pickle) (csv?) and object based methods
    4. Convert 3. to OT-2 scripts
    5. Add methods for calculations (especially initial rates)
        1. Rate method with Tmax attribute
        2. Constants methods
    6. Thin-layer for PyEnzyme for 2-3 interfaces (Input data, output/raw data)